In [1]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import mmcv
import torch
from mmcv import Config
from mmcls.datasets.pipelines import Compose
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.gridspec as gridspec
from PIL import Image
import heapq

In [2]:
configCAM = '../CAMModels/resnet/compCars_Original/resnet50_b128x2_compcars-original-split.py'
cfg = Config.fromfile(configCAM)

In [4]:
cfg.data.test

{'type': 'CompCars',
 'data_prefix': 'data/CompCars_sv_original_split/val',
 'ann_file': 'data/CompCars_sv_original_split/meta/val.txt',
 'pipeline': [{'type': 'LoadImageFromFile'},
  {'type': 'Resize', 'size': (256, -1)},
  {'type': 'CenterCrop', 'crop_size': 224},
  {'type': 'Normalize',
   'mean': [123.675, 116.28, 103.53],
   'std': [58.395, 57.12, 57.375],
   'to_rgb': True},
  {'type': 'ImageToTensor', 'keys': ['img']},
  {'type': 'Collect', 'keys': ['img']}]}

In [7]:
from mmcv.utils import Registry
DATASETS = Registry('dataset', scope='mmcls')

In [9]:
DATASETS

Registry(name=dataset, items={})